In [7]:
import matplotlib.pyplot as plt
from matplotlib import style
import numpy as np
style.use ('ggplot')
import pandas 

filename = 'ModelData.csv' #Importation of Data 
trainingdata = pandas.read_csv(filename)
print(trainingdata.shape)
print(trainingdata)  


print(trainingdata)

X = trainingdata.values[:, :2]
Y = trainingdata.values[:, 8]
filename2 = 'Testdata.csv'
TTdata = pandas.read_csv(filename2)
XT = TTdata.values[:, :2]
YT = TTdata.values[:, :8]




class SVM:
    def __init__(self, visualization=True):    #An Intition class for the classifier 
        self.visualization = visualization
        self.colors = {1:'r',-1:'b'}
        if self.visualization:          #visualisation of the classifier framework on a grid
            self.fig = plt.figure()
            self.ax = self.fig.add_subplot(2,2,1)
            #self.ax.subplot(2, 2, i + 1)
            #self.ax.subplots_adjust(wspace=0.4, hspace=0.4)
    
            
    def train(self, fitdata):         #Building Model using the training data 
        fitdata = np.array([],[])
        self.fitdata = fitdata 
        alldata = []
        Group = {} #Population base on {w,b}
    

    
        transforms = [[1,-1],
                     [1,1],
                     [-1,1],
                      [-1,-1]]
                  
        for yi in self.fitdata:
            for featureset in self.fitdata[yi]:
                for feature in featureset:
                    alldata.append(feature)
            
        self.max_feature_value = X[:, 0].max() + 1    #Search processs
        self.min_feature_value = X[:, 0].min() - 1 
        alldata = None
        h=5
        x_min = X[:, 0].min() - 1
        x_max = X[:, 0].max() + 1
    
        y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
        xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                     np.arange(y_min, y_max, h))
     
    
        stepsize = [self.max_feature_value* 0.05]
    
        b_range = 5
        b_multiple = 5
        lastest_optimum = self.max_feature_value*10
    
        for step in stepsize:                           #Optimization process
            w = np.array([lastest_optimum,lastest_optimum])
            optimized = False
            while not optimized:
                for b in np.arange(-1*(self.max_feature_value*b_multiple), self.max_feature_value*b_multiple, step*b_multiple):
                    for transformation in transforms: 
                        wt = w*transformation
                        found_option = True
                  
                        for i in self.fitdata:
                            for xi in self.fitdata[i]:
                                yi=i
                                if not yi*(np.dot(wt,xi)+b) >=1:
                                    found_option = False
                                  #break
                            
                  
                        if found_option:
                              Group[np.linalg.norm(wt)] = [wt,b]
                  
                if w[0] < 0:
                          optimized = True 
                          print('Processed Optimized')
                  
                else:
                          w = w - step 
            norms = sorted ([n for n in Group]) #Sorted list of the magnitudes 
            Group_choice = Group[norms[0]]
            self.w = Group_choice[0]
            self.b = Group_choice[1]
            latest_optimum = Group_choice[0][0]+step*2
        
        for i in self.fitdata:
            for xi in self.fitdata[i]:
                yi=i
                print(xi,':',yi*(np.dot(self.w,xi)+self.b))
            
    def predict(self,features):                                                   #Prediction of unlabelled data vs the model built 
        classification = np.sign(np.dot(np.array(features), self.w)+self.b)
        if classification !=0 and self.visualization:
            
            self.ax.scatter(features[:,0], features[:,1], marker='*', c=self.colors[classification])
        
        return classification
    
    def visualize(self):                                             #Displays points of each classs
        for i, clf in enumerate((SVM().train)):
    
            # Plot the decision boundary. For that, we will assign a color to each
            # point in the mesh [x_min, m_max]x[y_min, y_max].
            self.ax.subplot(2, 2, i + 1)
            self.ax.subplots_adjust(wspace=0.4, hspace=0.4)

            Z = self.predict(np.c_[xx.ravel(), yy.ravel()])

            # Put the result into a color plot
            Z = Z.reshape(xx.shape)
            self.ax.contourf(xx, yy, Z, cmap=plt.cm.Paired, alpha=0.8)

            # Plot also the training points
            self.ax.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.Paired)
            
            self.ax.xlim(xx.min(), xx.max())
            self.ax.ylim(yy.min(), yy.max())
            self.ax.xticks(())
            self.ax.yticks(())
            
        #[[self.ax.scatter(X[:, 0], X[:, 1],color=self.colors[i])for x in labels[i]] for i in labels]
        
                         
        def hyperplane(x,w,b,c): #Gives the hyperplane value  c=X.W+b , decision boundary, positive or negative,for the graph
                  calc = (-w[0]*x-b+c)/w[1]
                  return calc
                  
        datarange = (self.min_feature_value*9.0,self.max_feature_value*1.1) 
        hyper_x_max = datarange[1]
        hyper_x_min = datarange[0]
                  
                  
        postivesv1 = hyperplane(hyper_x_min, self.w, self.b, 1)#positive support vector, is going to be y, (w.X+b) = 1
        postivesv2 = hyperplane(hyper_x_max, self.w, self.b, 1)#second point       
        self.ax.plot([hyper_x_min,hyper_x_max], [postivesv1, postivesv2], 'k') 
                  
        negativesv1 = hyperplane(hyper_x_min, self.w, self.b, -1)#negative support vector, (W.X+b) = -1
        negativesv2 = hyperplane(hyper_x_max, self.w, self.b, -1)#second point       
        self.ax.plot([hyper_x_min,hyper_x_max], [negativesv1, negativesv2], 'k')
                  
        decisionsv1 = hyperplane(hyper_x_min, self.w, self.b, 0)#decision margin, (W.X+b) = 0       
        decisionsv2 = hyperplane(hyper_x_max, self.w, self.b, 0)#second point       
        self.ax.plot([hyper_x_min,hyper_x_max], [decisionsv1, decisionsv2], 'k--')   
        plt.show()
                  
labels = {-1:X[:,0], 1:X[:,1]}                 
classifier = SVM.train(fitdata = labels, self = classifier)  
trainingdata.astype(int)
SVM.visualize(classifier)
#Dset = {-1:trainingdata[:,1:7],1:trainingdata[:,8]}
#labels = {-1:X[:,0], 1:X[:,1]}
#fit = classifier.train(fitdata = labels)
#classifier.visualize()
                  
predictTest = TTdata.values[:, :2] 

for f in predictTest:
    SVM.predict(f)
    
    
SVM.visualize(classifier)
                  
    
                  



            

(1367, 9)
       3  139  54   0  0.1  25.6  0.402  22  1
0      6  119  50  22  176  27.1  1.318  33  1
1      9  184  85  15    0  30.0  1.213  49  1
2     12   92  62   7  258  27.6  0.926  44  1
3      1  113  64  35    0  33.6  0.543  21  1
4     11  155  76  28  150  33.3  1.353  51  1
5      3  141   0   0    0  30.0  0.761  27  1
6      4  123  62   0    0  32.0  0.226  35  1
7      0  138   0   0    0  36.3  0.933  25  1
8      2  146   0   0    0  27.5  0.240  28  1
9     10  101  86  37    0  45.6  1.136  38  1
10     7  106  60  24    0  26.5  0.296  29  1
11     0  146  70   0    0  37.9  0.334  28  1
12     7  161  86   0    0  30.4  0.165  47  1
13     2  108  80   0    0  27.0  0.259  52  1
14     1  119  86  39  220  45.6  0.808  29  1
15     0  107  62  30   74  36.6  0.757  25  1
16     2  128  78  37  182  43.3  1.224  31  1
17     1  128  48  45  194  40.5  0.613  24  1
18     2  146  70  38  360  28.0  0.337  29  1
19    14  100  78  25  184  36.6  0.412  46  1
20 

AttributeError: 'NoneType' object has no attribute 'fitdata'